Traditionally, models used for object detection require labeled image datasets for training, and are limited to detecting the set of classes from the training data.

Zero-shot object detection, however, is supported by the OWL-ViT model which uses a different approach. OWL-ViT is an open-vocabulary object detector. It means that it can detect objects in images based on free-text queries without the need to fine-tune the model on labeled datasets.

OWL-ViT leverages multi-modal representations to perform open-vocabulary detection. It combines CLIP with lightweight object classification and localization heads. Open-vocabulary detection is achieved by embedding free-text queries with the text encoder of CLIP and using them as input to the object classification and localization heads. These heads associate images and their corresponding textual descriptions, and ViT processes image patches as inputs. The authors of OWL-ViT first trained CLIP from scratch and then fine-tuned OWL-ViT end to end on standard object detection datasets using a bipartite matching loss. With this approach, the model can detect objects based on textual descriptions without prior training on labeled datasets.

This guide shows how to use OWL-ViT:
1. to detect objects based on text prompts
2. for batch object detection
3. for image-guided object detection

# Libraries

In [1]:
pip install -q transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import pipeline
import numpy as np
from PIL import Image


# Zero-shot object detection pipeline

In [4]:
# The simplest way to try out inference with OWL-ViT is to use it in a pipeline()
# Instantiate a pipeline for zero-shot object detection from a checkpoint on the Hugging Face Hub
checkpoint = "google/owlv2-base-patch16-ensemble"
detector = pipeline(model=checkpoint, task="zero-shot-object-detection")

config.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/620M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [ ]:
# Choose an image you’d like to detect objects in. 
# Here we’ll use the image of astronaut Eileen Collins...
# ...part of the NASA Great Images dataset
image = skimage.data.astronaut()
image = Image.fromarray(np.uint8(image)).convert("RGB")

image